# To execute before  running the slides

In [ ]:
import unittest

def apply_jupyter_patch():
    """Monkey patch unittest to be able to run it in the notebook"""
    def jupyter_unittest_main(**kwargs):
        if "argv" not in kwargs:
            kwargs["argv"] = ['ignored']
        kwargs["exit"] = False
        jupyter_unittest_main._original(**kwargs)

    if unittest.main.__module__ != "unittest.main":
        # Restiture the previous state, in case
        unittest.main = unittest.main._original

    # Apply the patch
    jupyter_unittest_main._original = unittest.main
    unittest.main = jupyter_unittest_main

apply_jupyter_patch()


def polynom(a, b, c):
    """The function that will be tested."""
    delta = (b**2.0) - 4.0 * a * c
    solutions = []
    if delta > 0:
        solutions.append((-b + (delta**0.5)) / (2.0 * a))
        solutions.append((-b - (delta**0.5)) / (2.0 * a))
    elif delta == 0:
        solutions.append(-b / (2.0 * a))
    return solutions


try:
    from PyQt5 import Qt

    qapp = Qt.QApplication.instance()
    if Qt.QApplication.instance() is None:
        qapp = Qt.QApplication([])

    class PolynomSolver(Qt.QMainWindow):
    
        def __init__(self, parent=None):
            super(PolynomSolver, self).__init__(parent=parent)
            self.initGui()
    
        def initGui(self):
            self.setWindowTitle("Polygon Solver")
            self._inputLine = Qt.QLineEdit(self)
            self._processButton = Qt.QPushButton(self)
            self._processButton.setText(u"Solve ax² + bx + c = 0")
            self._processButton.clicked.connect(self.processing)
            self._resultWidget = Qt.QLabel(self)
    
            widget = Qt.QWidget()
            layout = Qt.QFormLayout(widget)
            layout.addRow("Coefs a b c:", self._inputLine)
            layout.addRow("Solutions:", self._resultWidget)
            layout.addRow(self._processButton)
            self.setCentralWidget(widget)
    
        def getCoefs(self):
            text = self._inputLine.text()
            data = [float(i) for i in text.split()]
            a, b, c = data
            return a, b, c
    
        def processing(self):
            try:
                a, b, c = self.getCoefs()
            except Exception as e:
                Qt.QMessageBox.critical(self, "Error while reaching polygon coefs", str(e))
                return
            try:
                result = polynom(a, b, c)
            except Exception as e:
                Qt.QMessageBox.critical(self, "Error while computing the polygon solution", str(e))
                return
    
            if len(result) == 0:
                text = "No solution"
            else:
                text = ["%0.3f" % x for x in result]
                text = " ".join(text)
            self._resultWidget.setText(text)
except ImportError as e:
    print(str(e))

Testing
=======

- Introduction
- Python `unittest` module
- Estimate tests' quality
- Continuous integration

# What is it?

- Part of the software quality
- A task consisting of checking that the **program** is working as expected
- Manually written **tests** which can be automatically executed

<img src="img/test.svg" style="height:50%;margin-left:auto;margin-right:auto;padding:2em;">

# Presenter Notes

- A test injects input to the program, and checks output
- It answers if the code is valid or not (for a specific usecase)

# Different methodologies

- Test-driven development: Always and before anything else

<img src="img/ttd-workflow.svg" style="height:50%;margin-left:auto;margin-right:auto;padding:2em;">

- Harry J.W. Percival (2014). [Test-Driven Development with Python. O'Reilly](https://www.oreilly.com/library/view/test-driven-development-with/9781449365141/)

# Why testing?

| Benefits                                | Disadvantage                                |
|-----------------------------------------|---------------------------------------------|
| Find problems early                     | Extra work (to write and execute)           |
| Globally reduce the cost                | Maintain test environments                  |
| To validate the code to specifications  | Does not mean it's bug-free                 |
| Safer to changes of the code with       | More difficult to change the code behaviour |
| Improve the software design             | &nbsp;                                      |
| It's part of documentation and examples | &nbsp;                                      |


# Presenter Notes

- 30% percent of the time of a project
- Cost reduction: If you find a problem late (at deployment for example) the cost can be very hight
- Automated tests (in CI) reduce the cost of execution, and help code review
- Having the structure set-up for testing encourages writing tests


# What kinds of tests?

- <span style="color:#ee5aa0">**Unit tests**</span>: Tests independant pieces of code
- <span style="color:#19bdcd">**Integration tests**</span>: Tests components together
- <span style="color:#1aac5b">**System tests**</span>: Tests a completely integrated application
- <span style="color:#b8b800">**Acceptance tests**</span>: Tests the application with the customer

<img src="img/test-kind.svg" style="height:50%;margin-left:auto;margin-right:auto;padding:0em;">

# Presenter Notes

The test pyramid is a concept developed by Mike Cohn, described in his book "Succeeding with Agile"

- Unit tests (dev point of view, fast, low cost)
- Integration tests
- System tests
- Acceptance tests (customer point of view, but slow, and expensive, can't be automated)

- Cost: unit << integration (not always true) << system
- Fast to execute: unit >> integration >> system


# Where to put the tests?

Separate tests from the source code:

- Run the test from the command line.
- Separate tests and code distributing.
- [...](https://docs.python.org/3/library/unittest.html#organizing-test-code)

Folder structure:

- In a separate `test/` folder.
- In `test` sub-packages in each Python package/sub-package,
  so that tests remain close to the source code.
  Tests are installed with the package and can be run from the installation.
- A `test_*.py` for each module and script (an more if needed).
- Consider separating tests that are long to run from the others.


# Where to put the tests?

- `project`
  - `setup.py`
  - `run_tests.py`
  - `package/`
    - `__init__.py`
    - `module1.py`
    - `test/`
      - `__init__.py`
      - `test_module1.py`
    - `subpackage/`
      - `__init__.py`
      - `module1.py`
      - `module2.py`
      - `test/`
        - `__init__.py`
        - `test_module1.py`
        - `test_module2.py`

# `unittest` Python module

[unittest](https://docs.python.org/3/library/unittest.html) is the default Python module for testing.

It provides features to:

- Write tests
- Discover tests
- Run those tests

Other frameworks exists:

- [pytest](http://pytest.org/)

# Write and run tests

The classe `unittest.TestCase` is the base class for writting tests for
Python code.

The function `unittest.main()` provides a command line interface to
discover and run the tests.

In [ ]:
import unittest

class TestMyTestCase(unittest.TestCase):

    def test_my_test(self):
        # Code to test
        a = round(3.1415)
        # Expected result
        b = 3
        self.assertEqual(a, b, msg="")

if __name__ == "__main__":
    unittest.main()

# Assertion functions

- Argument(s) to compare/evaluate.
- An additional error message.

- `assertEqual(a, b)` checks that `a == b`
- `assertNotEqual(a, b)` checks that `a != b`
- `assertTrue(x)` checks that `bool(x) is True`
- `assertFalse(x)`checks that `bool(x) is False`
- `assertIs(a, b)` checks that `a is b`
- `assertIsNone(x)` checks that `x is None`
- `assertIn(a, b)` checks that `a in b`
- `assertIsInstance(a, b)` checks that `isinstance(a, b)`

There's more, see [unittest TestCase documentation](https://docs.python.org/3/library/unittest.html#unittest.TestCase>)
or [Numpy testing documentation](http://docs.scipy.org/doc/numpy/reference/routines.testing.html).

# Example

Test the `polynom` function provided in the `pypolynom` sample project.

It solves the equation $ax^2 + bx + c = 0$.

In [ ]:
import unittest

class TestPolynom(unittest.TestCase):

    def test_0_roots(self):
        result = polynom(2, 0, 1)
        self.assertEqual(len(result), 0)

    def test_1_root(self):
        result = polynom(2, 0, 0)
        self.assertEqual(len(result), 1)
        self.assertEqual(result, [0])

    def test_2_root(self):
        result = polynom(4, 0, -4)
        self.assertEqual(len(result), 2)
        self.assertEqual(set(result), set([-1, 1]))

if __name__ == "__main__":
    unittest.main(defaultTest="TestPolynom")
    # unittest.main(verbosity=2, defaultTest="TestPolynom")

# Run from command line arguments

Auto discover tests of the current path

Running a specific `TestCase`:

Running a specific test method:

# Fixture

Tests might need to share some common initialisation/finalisation (e.g., create a temporary directory).

This can be implemented in ``setUp`` and ``tearDown`` methods of ``TestCase``.
Those methods are called before and after each test.

In [ ]:
class TestCaseWithFixture(unittest.TestCase):

    def setUp(self):
        self.file = open("img/test-pyramid.svg", "rb")
        print("open file")

    def tearDown(self):
        self.file.close()
        print("close file")

    def test_1(self):
        foo = self.file.read()
        # do some test on foo
        print("test 1")

    def test_2(self):
        foo = self.file.read()
        # do some test on foo
        print("test 2")

if __name__ == "__main__":
    unittest.main(defaultTest='TestCaseWithFixture')

# Testing exception

In [ ]:
class TestPolynom(unittest.TestCase):

    def test_argument_error(self):
        try:
            polynom(0, 0, 0)
            self.fail()
        except ZeroDivisionError:
            self.assertTrue(True)

    def test_argument_error__better_way(self):
        with self.assertRaises(ZeroDivisionError):
            result = polynom(0, 0, 0)

if __name__ == "__main__":
    unittest.main(defaultTest='TestPolynom')

`TestCase.assertRaisesRegexp` also checks the message of the exception.

# Parametric tests

Running the same test with multiple values

Problems:

- The first failure stops the test, remaining test values are not processed.
- There is no information on the value for which the test has failed.

In [ ]:
class TestPolynom(unittest.TestCase):

    TESTCASES = {
        (2, 0, 1): [],
        (2, 0, 0): [0],
        (4, 0, -4): [1, -1]
    }

    def test_all(self):
        for arguments, expected in self.TESTCASES.items():
            self.assertEqual(polynom(*arguments), expected)

    def test_all__better_way(self):
        for arguments, expected in self.TESTCASES.items():
            with self.subTest(arguments=arguments, expected=expected):
                self.assertEqual(polynom(*arguments), expected)

if __name__ == "__main__":
    unittest.main(defaultTest='TestPolynom')

# Class fixture

In [ ]:
class TestSample(unittest.TestCase):

    @classmethod
    def setUpClass(cls):
        # Called before all the tests of this class
        pass

    @classmethod
    def tearDownClass(cls):
        # Called after all the tests of this class
        pass

# Module fixture


In [ ]:
def setUpModule():
    # Called before all the tests of this module
    pass

def tearDownModule():
    # Called after all the tests of this module
    pass

# Skipping tests

If tests requires a specific OS, device, library...

In [ ]:
import unittest, os, sys

def is_gui_available():
    # Is there a display
    if sys.platform.startswith('linux'):
        if os.environ.get('DISPLAY', '') == '':
            return False
    # Is there the optional library
    try:
        import PyQt8
    except:
        return False
    return True

@unittest.skipUnless(is_gui_available(), 'GUI not available')
class TestPolynomGui(unittest.TestCase):

    def setUp(self):
        if not is_gui_available():
            self.skipTest('GUI not available')

    def test_1(self):
        if not is_gui_available():
            self.skipTest('GUI not available')

    @unittest.skipUnless(is_gui_available() is None, 'GUI not available')
    def test_2(self):
        pass

if __name__ == "__main__":
    unittest.main(defaultTest='TestPolynomGui')

# Test numpy

Numpy provides modules for unittests. See the [Numpy testing documentation](http://docs.scipy.org/doc/numpy/reference/routines.testing.html).

In [ ]:
import numpy

class TestNumpyArray(unittest.TestCase):

    def setUp(self):
        self.data1 = numpy.array([1, 2, 3, 4, 5, 6, 7])
        self.data2 = numpy.array([1, 2, 3, 4, 5, 6, 7.00001])

    # def test_equal__cant_work(self):
    #     self.assertEqual(self.data1, self.data2)
    #     self.assertTrue((self.data1 == self.data2).all())

    def test_equal(self):
        self.assertTrue(numpy.allclose(self.data1, self.data2, atol=0.0001))

    def test_equal__even_better(self):
        numpy.testing.assert_allclose(self.data1, self.data2, atol=0.0001)

if __name__ == "__main__":
    unittest.main(defaultTest='TestNumpyArray')

# Test resources

How to handle test data?

Need to separate (possibly huge) test data from python package.

Download test data and store it in a temporary directory during the tests if not available.

Example: [silx.utils.ExternalResources](https://github.com/silx-kit/silx/blob/master/silx/utils/utilstest.py)

# QTest

For GUI based on `PyQt`, `PySide` it is possible to use Qt's [QTest](http://doc.qt.io/qt-5/qtest.html).

It provides the basic functionalities for GUI testing.
It allows to send keyboard and mouse events to widgets.

In [ ]:
from PyQt5.QtTest import QTest

class TestPolynomGui(unittest.TestCase):

    def test_type_and_process(self):
        widget = PolynomSolver()
        QTest.qWaitForWindowExposed(widget)
        QTest.keyClicks(widget._inputLine, '2.000 0 -1', delay=100)  # Wait 100ms
        QTest.mouseClick(widget._processButton, Qt.Qt.LeftButton, pos=Qt.QPoint(1, 1))
        self.assertEqual(widget._resultWidget.text(), "0.707 -0.707")

if __name__ == "__main__":
    unittest.main(defaultTest='TestPolynomGui')

Tighly coupled with the code it tests.
It needs to know the widget's instance and hard coded position of mouse events.

# Chaining tests

How-to run tests from many ``TestCase`` and many files at once:

- Explicit:
    Full control, boilerplate code.

- Automatic:
    No control

- Mixing approach


The [TestSuite](https://docs.python.org/3/library/unittest.html#unittest.TestSuite) class aggregates test cases and test suites through:

- Allow to test specific use cases
- Full control of the test sequence
- But requires some boilerplate code

# Chaining tests example

In [ ]:
def suite_without_gui():
    loadTests = unittest.defaultTestLoader.loadTestsFromTestCase
    suite = unittest.TestSuite()
    suite.addTest(loadTests(TestPolynom))
    return suite

def suite_with_gui():
    loadTests = unittest.defaultTestLoader.loadTestsFromTestCase
    suite = unittest.TestSuite()
    suite.addTest(suite_without_gui())
    suite.addTest(loadTests(TestPolynomGui))
    return suite

if __name__ == "__main__":
    # unittest.main(defaultTest='suite_without_gui')
    unittest.main(defaultTest='suite_with_gui')

# Estimate tests' quality

Using [`coverage`](https://coverage.readthedocs.org) to gather coverage statistics while running the tests (`pip install coverage`).

# Estimate tests' quality

Execute the tests and generate an output file per module with annotations per lines.

# Continuous integration

Automatically testing a software for each changes applied to the source code.

Benefits:

- Be aware of problems early
    - Before merging a change on the code
    - On third-party library update (sometimes before the release)
    - Reduce the cost in case of problem
- Improve contributions and team work

Costs:

- Set-up and maintenance
- Test needs to be automated


# Continuous integration

- [Travis-CI](https://travis-ci.org/) (Linux and MacOS), [AppVeyor](http://www.appveyor.com/) (Windows), gitlab-CI (https://gitlab.esrf.fr)...
- A `.yml` file to describing environment, build, installation, test process

<img src="img/ci-workflow.svg" style="width:40%;margin-left:auto;margin-right:auto;padding:0em;">

# Continuous integration: Configuration

Example of configuration with Travis

# Sum-up

- A coverage of most cases is a good start (80/20%).
- Tests should be done early, to identify design problems, and to improve team work.
- The amount of work must not be under estimate. Designing good test is about 1/3 of resources a project.
- Aiming at exhaustive tests is not needed and utopic.
- To be tested, an application have to be architectured in this way.
- Continuous integration is particularly useful to prevent regressions, and help contributions.
- Next step: Continuous deployment.